In [31]:
import pandas as pd

In [61]:
pop_ori = pd.read_csv('./ACS_5yrs_2020/ACSDT5Y2020.csv', header=1)
# land = pd.read_csv('./2020_Gaz_zcta_national.csv')
land = pd.read_csv('~/Downloads/2020_Gaz_zcta_national.csv')

In [74]:
'''
Note: how to check a value exists in a Pandas series or not
if just do:
     x in SOME_SERIES
    this is checking whether x is in "the index" of the series, not value
'''
land.GEOID.values

array([  601,   602,   603, ..., 99926, 99927, 99929])

In [75]:
land.head()

,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,601,166659744,799292,64.348,0.309,18.180555,-66.749961
1,602,79307538,4428428,30.621,1.710,18.361945,-67.175597
2,603,81887203,181412,31.617,0.070,18.455183,-67.119887
3,606,109579950,12487,42.309,0.005,18.158327,-66.932928
4,610,93013430,4172059,35.913,1.611,18.294032,-67.127156


In [76]:
# get only GEOID and ALAND, exclude other fields I'm not interested
land = land[['GEOID', 'ALAND_SQMI']]
# land.head()

In [87]:
'''
quick pre processing
    - rename
    - drop useless columns
    - remove ZCTA string from every ZCTA cell
    - change type of ZCTA to type int
'''
pop = pop_ori.rename({'Estimate!!Total': 'population', 'Geographic Area Name': 'ZCTA'}, axis=1)
pop = pop.drop(['Margin of Error!!Total', 'id'], axis=1)
pop['ZCTA'] = pop['ZCTA'].map(lambda x: x.lstrip('ZCTA5 '))
pop['ZCTA'] = pop['ZCTA'].astype(int)
pop.head()


,population,ZCTA
0,469,89010
1,2068,89019
2,10692,89060
3,5537,89061
4,1244,89439


In [88]:
pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1769 entries, 0 to 1768
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   population  1769 non-null   int64
 1   ZCTA        1769 non-null   int64
dtypes: int64(2)
memory usage: 27.8 KB


In [38]:
# pop.describe()

In [90]:
'''
Devil in the detail: need to change ZCTA type from str object to int in pop dataframe
'''
pop.set_index('ZCTA').join(land.set_index('GEOID'))
# pop.join(land.set_index('GEOID'), on='ZCTA')

,population,ALAND_SQMI
ZCTA,,
89010,469,404.233
89019,2068,478.048
89060,10692,462.549
89061,5537,72.710
89439,1244,14.579
...,...,...
96148,679,1.360
96150,30369,123.542
96155,0,37.043


In [77]:
# join pop and land dataframe
tb = pop.set_index('ZCTA').join(land.set_index('GEOID'))
# tb.head()

In [78]:
tb

,population,ALAND_SQMI
ZCTA,,
89010,469,NaN
89019,2068,NaN
89060,10692,NaN
89061,5537,NaN
89439,1244,NaN
...,...,...
96148,679,NaN
96150,30369,NaN
96155,0,NaN


In [40]:
pop_dens = tb.copy()
pop_dens['pop_density'] = pop_dens['population'] / pop_dens['ALAND_SQMI']
pop_dens = pop_dens.rename({'ALAND_SQMI': 'area (sq mi)'}, axis=1)
pop_dens.head()

,population,area (sq mi),pop_density
ZCTA,,,
89010,469,NaN,NaN
89019,2068,NaN,NaN
89060,10692,NaN,NaN
89061,5537,NaN,NaN
89439,1244,NaN,NaN


In [41]:
pop_dens

,population,area (sq mi),pop_density
ZCTA,,,
89010,469,NaN,NaN
89019,2068,NaN,NaN
89060,10692,NaN,NaN
89061,5537,NaN,NaN
89439,1244,NaN,NaN
...,...,...,...
96148,679,NaN,NaN
96150,30369,NaN,NaN
96155,0,NaN,NaN


In [42]:
pop_dens.isnull().any()

population      False
area (sq mi)     True
pop_density      True
dtype: bool

In [43]:
pop_dens.describe()

,population,area (sq mi),pop_density
count,1769.000000,0.0,0.0
mean,22248.884681,NaN,NaN
std,22598.308562,NaN,NaN
min,0.000000,NaN,NaN
25%,1513.000000,NaN,NaN
50%,15974.000000,NaN,NaN
75%,36820.000000,NaN,NaN
max,110750.000000,NaN,NaN


In [39]:
# inspect the max density region. Turned out to be a mini zone in DC
pop_dens.loc[pop_dens['pop_density'] == max(pop_dens['pop_density'])]

,population,area (sq mi),pop_density
zipcode,,,
20052,470,0.003,156666.666667


In [41]:
'''Export to .csv'''
pop_dens.to_csv('./population_density_zcta.csv')